In [ ]:
"""🚀 MARIA HELENA TRADING BOT - LSTM TRAINERGoogle Colab Notebook (Convert to .ipynb)"""# ============================================================

In [ ]:
# ============================================================import numpy as npimport pandas as pdimport tensorflow as tffrom tensorflow import kerasfrom tensorflow.keras.models import Sequentialfrom tensorflow.keras.layers import LSTM, Dense, Dropoutfrom sklearn.preprocessing import MinMaxScalerfrom sklearn.metrics import mean_squared_error, mean_absolute_errorimport matplotlib.pyplot as pltimport warningswarnings.filterwarnings('ignore')print("✅ Bibliotecas importadas!")# ============================================================

In [ ]:
# ============================================================# Opção A: Do GitHuburl = "https://raw.githubusercontent.com/seu-usuario/maria-helena/main/bitcoin_training_data.csv"df = pd.read_csv(url)# Opção B: Do Google Drive# from google.colab import drive# drive.mount('/content/drive')# df = pd.read_csv('/content/drive/My Drive/maria-helena/bitcoin_training_data.csv')print(f"✅ {len(df)} candles carregados!")print(df.head())# ============================================================

In [ ]:
# ============================================================# Usar close price para treinardata = df['close'].values.reshape(-1, 1)# Normalizarscaler = MinMaxScaler(feature_range=(0, 1))scaled_data = scaler.fit_transform(data)# Criar sequences de treino (60 dias anteriores -> próximo dia)lookback = 60X_train = []y_train = []for i in range(lookback, len(scaled_data)):    X_train.append(scaled_data[i-lookback:i, 0])    y_train.append(scaled_data[i, 0])X_train = np.array(X_train)y_train = np.array(y_train)# Reshape para LSTM [samples, timesteps, features]X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))print(f"✅ Dados preparados!")print(f"   X_train shape: {X_train.shape}")print(f"   y_train shape: {y_train.shape}")# ============================================================

In [ ]:
# ============================================================model = Sequential([    LSTM(50, activation='relu', input_shape=(lookback, 1)),    Dropout(0.2),    LSTM(50, activation='relu'),    Dropout(0.2),    Dense(25, activation='relu'),    Dense(1)])model.compile(optimizer='adam', loss='mse', metrics=['mae'])model.summary()print("✅ Modelo criado!")# ============================================================

In [ ]:
# ============================================================history = model.fit(    X_train, y_train,    epochs=50,    batch_size=32,    validation_split=0.2,    verbose=1)print("✅ Modelo treinado!")# ============================================================

In [ ]:
# ============================================================plt.figure(figsize=(12, 4))plt.plot(history.history['loss'], label='Train Loss')plt.plot(history.history['val_loss'], label='Validation Loss')plt.title('Model Loss')plt.xlabel('Epoch')plt.ylabel('Loss')plt.legend()plt.grid(True)plt.show()# ============================================================

In [ ]:
# ============================================================# Predições no conjunto de treinotrain_predict = model.predict(X_train)train_predict = scaler.inverse_transform(train_predict)y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))# Calcular RMSEtrain_rmse = np.sqrt(mean_squared_error(y_train_actual, train_predict))print(f"Train RMSE: ${train_rmse:.2f}")# ============================================================

In [ ]:
# ============================================================# Usar últimos 60 candles pra prever o próximolast_60_days = data[-60:]last_60_days_scaled = scaler.transform(last_60_days)X_test = np.array([last_60_days_scaled])X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))next_price = model.predict(X_test)next_price = scaler.inverse_transform(next_price)print(f"🚀 Preço predito: ${next_price[0][0]:.2f}")print(f"💰 Preço atual: ${data[-1][0]:.2f}")# ============================================================

In [ ]:
# ============================================================model.save('maria_helena_lstm_model.h5')print("✅ Modelo salvo como maria_helena_lstm_model.h5")# ============================================================

In [ ]:
# ============================================================# Copiar arquivo do Colab pra GitHub/Drive# ! cp maria_helena_lstm_model.h5 /content/drive/My\ Drive/print("✅ Salve o arquivo: maria_helena_lstm_model.h5")